<a href="https://colab.research.google.com/github/NityaVerma19/Customer-Purchase-Decision-/blob/master/Consumer_behaviour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split


#Generating a dataset

### Product Info

In [67]:
#Green Packaging - yes/no

def generate_random_series(words, length, seed = 42):
    random.seed(seed)
    series = [random.choice(words) for _ in range(length)]
    return series

word_list = ['Yes', 'No']
series_length = 500

green_pack = generate_random_series(word_list, series_length, seed = 42)


In [68]:
#purchase decision - yes/no

def generate_random_series(length, seed = 42):
    random.seed(seed
    )
    return [random.choice([0, 1]) for _ in range(length)]

length = 500
purchase = generate_random_series(length, seed = 42)


In [69]:
#type of product - 'Beverage', 'Food' , 'Beauty', 'Personal hygiene', 'Paper products' , 'Clothing'


def generate_random_series(words, length, seed = 42):
    random.seed(seed)
    series = [random.choice(words) for _ in range(length)]
    return series

word_list = ['Beverage', 'Food' , 'Beauty', 'Personal hygiene', 'Paper products' , 'Clothing']
series_length = 500

product = generate_random_series(word_list, series_length, seed = 42)


In [70]:
#Price
def generate_random_series(lower_bound, upper_bound, count, seed = 42):
    random.seed(seed)
    random_numbers = np.random.randint(lower_bound, upper_bound + 1, count)
    return pd.Series(random_numbers)


lower_bound = 500
upper_bound = 5000
count = 500  # Number of random numbers to generate
price = generate_random_series(lower_bound, upper_bound, count, seed = 42)



### Customer Info

In [71]:
#Age

def generate_random_series(lower_bound, upper_bound, count, seed = 42):
    random.seed(seed)
    random_numbers = np.random.randint(lower_bound, upper_bound + 1, count)
    return pd.Series(random_numbers)


lower_bound = 18
upper_bound = 50
count = 500  # Number of random numbers to generate
age = generate_random_series(lower_bound, upper_bound, count, seed = 42)



In [72]:
#Gender


def generate_random_series(words, length, seed = 42):
    random.seed(seed)
    series = [random.choice(words) for _ in range(length)]
    return series

word_list = ['M', 'F']
series_length = 500

gender = generate_random_series(word_list, series_length, seed = 42)


In [73]:
#Income level - 15k to 150k

def generate_random_series(lower_bound, upper_bound, count, seed = 42):
    random.seed(seed)
    random_numbers = np.random.randint(lower_bound, upper_bound + 1, count)
    return pd.Series(random_numbers)


lower_bound = 15000
upper_bound = 150000
count = 500  # Number of random numbers to generate
income = generate_random_series(lower_bound, upper_bound, count, seed = 42)


In [84]:
#customer id


numbers_sequence = range(1, 500)
cust_id = pd.Series(numbers_sequence)


*Merging the series*

In [83]:
df = pd.DataFrame({'cust_id': cust_id,'product': product, 'price': price, 'green_pack': green_pack, 'age': age,'gender': gender, 'income': income, 'purchase': purchase })
df.head()

,cust_id,product,price,green_pack,age,gender,income,purchase
0,1.0,Clothing,1215,Yes,40,M,85238,0
1,2.0,Beverage,524,Yes,48,M,31089,0
2,3.0,Beverage,3695,No,43,F,139863,1
3,4.0,Clothing,2878,Yes,32,M,124897,0
4,5.0,Beauty,4344,Yes,48,M,65008,0


In [ ]:
# Downloading the dataframe

#df.to_csv('df.csv', index=False)

In [85]:
df.head()

,cust_id,product,price,green_pack,age,gender,income,purchase
0,1.0,Clothing,1215,Yes,40,M,85238,0
1,2.0,Beverage,524,Yes,48,M,31089,0
2,3.0,Beverage,3695,No,43,F,139863,1
3,4.0,Clothing,2878,Yes,32,M,124897,0
4,5.0,Beauty,4344,Yes,48,M,65008,0



---





# Logistic Regression


In [12]:
df.head()

,cust_id,product,price,green_pack,age,gender,income,purchase
0,1,Clothing,3961,Yes,48,M,28863,0
1,2,Beverage,3849,Yes,21,M,60866,0
2,3,Beverage,4950,No,34,F,97901,1
3,4,Clothing,4964,Yes,19,M,82031,0
4,5,Beauty,2833,Yes,21,M,81903,0


In [65]:
#Target variable
X = df.purchase.values

#Feature variables
features = ['product', 'price', 'green_pack', 'age', 'gender', 'income']
y = df[features]


In [66]:
logreg = LogisticRegression(solver='liblinear', random_state=0)


# fit the model
logreg.fit(X, y)

ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0.
 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.